# Quering a CSV and create a new CSV from results
## Read CSV

In [10]:
import pandas as pd

path = 'data/bvl-places-leipzig.csv'
dataframe = pd.read_csv(path)

## Custom Queries

Parameters:

- rows: Lists all row elements of the resulting table
- columns: Lists all column elements of the resulting table
- query dictionary: has to consist of a **query** function and it needs a **row** specifier and **col** specifier for correct item placement in the resulting table

In [14]:
rows = [
    'Gebäude aus Bildung/Freizeit/Kultur', 
    'Gebäude aus Dienstleistungen', 
    'Gebäude aus Gastronomie/Übernachtung', 
    'Gebäude aus Gesundheit/Soziales', 
    'Gebäude aus Recht/Verwaltung/Wirtschaft', 
    'Gebäude aus Verbände/Vereine', 
    'Gebäude aus Verkehr'
]
columns = [
    'Anzahl-mit-Eingangsbereich-vollständig-zugänglich-für-Rollstuhlfahrer',
    'Anzahl-mit-Eingangsbereich-eingeschränkt-zugänglich-für-Rollstuhlfahrer',
    'Anzahl-mit-Aufzug--für-Rollstuhlfahrer-voll-zugänglich',
    'Anzahl-mit-Personenaufzug-vorhanden',
    'Anzahl-mit-Toilette-vollständig-zugänglich-für-Rollstuhlfahrer',
    'Anzahl-mit-Toilette-eingeschränkt-zugänglich-für-Rollstuhlfahrer',
    'Anzahl-mit-Hilfen-für-Hörgeschädigte',
    'Anzahl-mit-Hilfen-für-Blinde-oder-Sehbehinderte-Menschen',
    'Anzahl-mit-speziellen-und-persönlichen-Hilfeleistungen-für-Menschen-mit-Behinderung',
    'Anzahl-mit-markierten-Behindertenparkplätzen'
]

def execute_query(q, df):
    return q(df)

def create_query_dicts(rows, columns, df):
    '''Helper method for creating all queries'''
    
    def create_queries(cat, idx):
        return [
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Eingangsbereich-rollstuhlgerecht'] == 'vollständig')]), 'row': rows[idx], 'col': columns[0]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Eingangsbereich-rollstuhlgerecht'] == 'teilweise')]), 'row': rows[idx], 'col': columns[1]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Personenaufzug-rollstuhlgerecht'] == 'ja')]), 'row': rows[idx], 'col': columns[2]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Personenaufzug-vorhanden'] == 'ja')]), 'row': rows[idx], 'col': columns[3]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Toilette-rollstuhlgerecht'] == 'vollständig')]), 'row': rows[idx], 'col': columns[4]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Toilette-rollstuhlgerecht'] == 'teilweise')]), 'row': rows[idx], 'col': columns[5]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Besondere-Hilfestellungen-f-Menschen-m-Hoerbehinderung-vorhanden'] == 'ja')]), 'row': rows[idx], 'col': columns[6]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Besondere-Hilfestellungen-f-Menschen-m-Seebhind-Blinde-vorhanden'] == 'ja')]), 'row': rows[idx], 'col': columns[7]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & (df['Allgemeine-Hilfestellungen-vor-Ort-vorhanden'] == 'ja')]), 'row': rows[idx], 'col': columns[8]},
            {'query': lambda df: len(df[(df['Kategorie'] == cat) & ( (df['Anzahl-Behindertenparkplaetze-v-Einrichtung'] > 0) | (df['Anzahl-Behindertenparkplaetze-auf-hauseigenem-Parkplatz'] > 0) )]), 'row': rows[idx], 'col': columns[9]}
        ]
    
    categories = ['Bildung', 'Dienst', 'Gastwirtschaft', 'Gesundheit', 'Recht', 'Verbände', 'Verkehr']
    dicts = []
    for idx, cat in enumerate(categories):
        dicts = dicts + create_queries(cat, idx)            
    return dicts

## Execution

In [12]:
query_dicts = create_query_dicts(rows, columns, dataframe)

assert len(query_dicts) == len(columns) * len(rows), 'To few queries. The resulting table is not goinig to be fully filled.'
result_dataframe = pd.DataFrame(index=rows, columns=columns)

for d in query_dicts:
    r = execute_query(d['query'], dataframe)
    result_dataframe.set_value(d['row'], d['col'], r)

result_dataframe

,Anzahl-mit-Eingangsbereich-vollständig-zugänglich-für-Rollstuhlfahrer,Anzahl-mit-Eingangsbereich-eingeschränkt-zugänglich-für-Rollstuhlfahrer,Anzahl-mit-Aufzug--für-Rollstuhlfahrer-voll-zugänglich,Anzahl-mit-Personenaufzug-vorhanden,Anzahl-mit-Toilette-vollständig-zugänglich-für-Rollstuhlfahrer,Anzahl-mit-Toilette-eingeschränkt-zugänglich-für-Rollstuhlfahrer,Anzahl-mit-Hilfen-für-Hörgeschädigte,Anzahl-mit-Hilfen-für-Blinde-oder-Sehbehinderte-Menschen,Anzahl-mit-speziellen-und-persönlichen-Hilfeleistungen-für-Menschen-mit-Behinderung,Anzahl-mit-markierten-Behindertenparkplätzen
Gebäude aus Bildung/Freizeit/Kultur,148,84,57,48,62,86,35,41,219,99
Gebäude aus Dienstleistungen,7,5,0,3,0,0,1,1,7,4
Gebäude aus Gastronomie/Übernachtung,54,28,18,33,12,45,1,28,79,29
Gebäude aus Gesundheit/Soziales,119,45,58,47,38,46,14,19,153,85
Gebäude aus Recht/Verwaltung/Wirtschaft,73,32,44,21,29,34,4,14,62,61
Gebäude aus Verbände/Vereine,24,8,12,6,2,20,1,2,48,14
Gebäude aus Verkehr,3,0,0,2,0,1,0,1,3,6


## Write to CSV

Specify an output path

In [13]:
output = 'data/result.csv'
result_dataframe.to_csv(output)